In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

---
**Booleans:**

In [13]:
df = spark.read.format('csv')\
.option('header', 'true')\
.option('inferSchema', 'true')\
.load('Book_Exercises/data/csv/2012-12-01.txt')
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



---
**Working with Booleans**

In [17]:
from pyspark.sql.functions import col
df.where(col("InvoiceNo") != 536365)\
.select("InvoiceNo", "Description")\
.show(5, False)

# df.where("InvoiceNo <> 536365").show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [21]:
from pyspark.sql.functions import instr

priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1

df.where(df.StockCode.isin("DOT"))\
.where(priceFilter | descripFilter)\
.select("unitPrice").show()

+---------+
|unitPrice|
+---------+
|   569.77|
|   607.49|
+---------+



In [22]:
from pyspark.sql.functions import instr

DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1

df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
.where("isExpensive")\
.select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



In [23]:
from pyspark.sql.functions import expr

df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))\
.where("isExpensive")\
.select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



---
**Working with Numbers:**

In [24]:
from pyspark.sql.functions import expr, pow
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [25]:
# USING selectExpr
df.selectExpr(
"CustomerId",
"(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [34]:
# ROUNDING and BROUNDING
#if we are inbetween:
#    round ==> round up
#    bround => round down
from pyspark.sql.functions import lit, round, bround
df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [39]:
# Finding the correlation between two columns
from pyspark.sql.functions import corr
df.stat.corr('Quantity', 'UnitPrice')

-0.04112314436835551

In [42]:
# Summary of numeric columns:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

There are a number of statistical functions available in the StatFunctions Package (accessible
using stat

In [48]:
# Approximate quantile of a column
colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", [0.25, 0.5, 0.75, 0.9], 0.05)

[1.65, 2.51, 4.21, 5.95]

In [50]:
#df.stat.crosstab("StockCode", "Quantity").show()
df.stat.freqItems(["StockCode", "Quantity"]).show()

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+



**we can also add a unique ID to each row by using the function
monotonically_increasing_id. This function generates a unique value for each row, starting
with 0:**

In [51]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show(2)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
+-----------------------------+
only showing top 2 rows



---
**Working with Strings**

In [55]:
# initcap function will capitalize every word in a given string when that word is separated from another by a space
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show(5)

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
+--------------------+
only showing top 5 rows



In [56]:
# UPPER and lower case:
from pyspark.sql.functions import lower, upper
df.select(col("Description"),
lower(col("Description")),
upper(lower(col("Description")))).show(2)

+--------------------+--------------------+-------------------------+
|         Description|  lower(Description)|upper(lower(Description))|
+--------------------+--------------------+-------------------------+
|WHITE HANGING HEA...|white hanging hea...|     WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern|      WHITE METAL LANTERN|
+--------------------+--------------------+-------------------------+
only showing top 2 rows



In [57]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
ltrim(lit(" HELLO ")).alias("ltrim"),
rtrim(lit(" HELLO ")).alias("rtrim"),
trim(lit(" HELLO ")).alias("trim"),
lpad(lit("HELLO"), 3, " ").alias("lp"),
rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)

+------+------+-----+---+----------+
| ltrim| rtrim| trim| lp|        rp|
+------+------+-----+---+----------+
|HELLO | HELLO|HELLO|HEL|HELLO     |
|HELLO | HELLO|HELLO|HEL|HELLO     |
+------+------+-----+---+----------+
only showing top 2 rows



___
**Working with dates and times**

In [5]:
from pyspark.sql.functions import current_date, current_timestamp
dataDF = spark.range(10)\
.withColumn('Today', current_date())\
.withColumn('Now', current_timestamp())

dataDF.printSchema()
dataDF.show()

root
 |-- id: long (nullable = false)
 |-- Today: date (nullable = false)
 |-- Now: timestamp (nullable = false)

+---+----------+--------------------+
| id|     Today|                 Now|
+---+----------+--------------------+
|  0|2021-10-24|2021-10-24 16:24:...|
|  1|2021-10-24|2021-10-24 16:24:...|
|  2|2021-10-24|2021-10-24 16:24:...|
|  3|2021-10-24|2021-10-24 16:24:...|
|  4|2021-10-24|2021-10-24 16:24:...|
|  5|2021-10-24|2021-10-24 16:24:...|
|  6|2021-10-24|2021-10-24 16:24:...|
|  7|2021-10-24|2021-10-24 16:24:...|
|  8|2021-10-24|2021-10-24 16:24:...|
|  9|2021-10-24|2021-10-24 16:24:...|
+---+----------+--------------------+



In [8]:
# Adding and subtracting dates:
from pyspark.sql.functions import date_add, date_sub
dataDF.select(date_sub(col('Today'), 5), date_add(col('Today'), 5)).show(2)

+------------------+------------------+
|date_sub(Today, 5)|date_add(Today, 5)|
+------------------+------------------+
|        2021-10-19|        2021-10-29|
|        2021-10-19|        2021-10-29|
+------------------+------------------+
only showing top 2 rows



In [12]:
# The difference between two dates in days using datediff
# The months_between gives the number of monthes between two dates

from pyspark.sql.functions import lit, datediff, months_between, to_date
dataDF.withColumn("week_ago", date_sub(col("today"), 7))\
.select(datediff(col("week_ago"), col("today"))).show(1)
dataDF.select(
to_date(lit("2016-01-01")).alias("start"),
to_date(lit("2017-05-22")).alias("end"))\
.select(months_between(col("start"), col("end"))).show(1)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



___
**Dealing with null data**

In [16]:
# Drop
df.na.drop("all", subset=["StockCode", "InvoiceNo"]).show(1)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row



In [19]:
# Fill string columns with a specific value
df.na.fill("all", subset=["StockCode", "InvoiceNo"]).show(1)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row



In [21]:
fill_cols_vals = {"StockCode": 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals).show(2)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



In [22]:
# Replace values with something else
df.na.replace([""], ["UNKNOWN"], "Description").show(2)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



---
**WORKING WITH COMPLEX TYPES**

* **Structs:** \
You can think of structs as DataFrames within DataFrames.

In [24]:
from pyspark.sql.functions import struct
complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

We now have a DataFrame with a column complex. We can query it just as we might another
DataFrame, the only difference is that we use a dot syntax to do so, or the column method
getField:

In [27]:
complexDF.select("complex.Description").show(2)
complexDF.select(col("complex").getField("Description")).show(2)
complexDF.select("complex.*").show(2)

+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
+--------------------+
only showing top 2 rows

+--------------------+
| complex.Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
+--------------------+
only showing top 2 rows

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
+--------------------+---------+
only showing top 2 rows



___
* **ARRAYS** 

**1) Split:**
Splitting words

In [28]:
from pyspark.sql.functions import split
df.select(split(col('Description'), " ")).show(2)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



In [33]:
df.select(split(col('Description'), " ").alias("array_col"))\
         .selectExpr("array_col[0]", "array_col[1]").show(2)

+------------+------------+
|array_col[0]|array_col[1]|
+------------+------------+
|       WHITE|     HANGING|
|       WHITE|       METAL|
+------------+------------+
only showing top 2 rows



**2) Array length:**

In [42]:
from pyspark.sql.functions import size
splits = split(col('Description'), ' ')
df.select(size(splits).alias('array_length')).show(2)

+------------+
|array_length|
+------------+
|           5|
|           3|
+------------+
only showing top 2 rows



**3) array_contains:**
We can also see whether this array contains a value

In [47]:
# array_contains(column, word to search)

from pyspark.sql.functions import array_contains
splits = split(col("Description"), " ")
df.select(array_contains(splits, "WHITE").alias('Contains')).show(2)

+--------+
|Contains|
+--------+
|    true|
|    true|
+--------+
only showing top 2 rows



**3) explode** 
The explode function takes a column that consists of arrays and creates one row (with the rest of
the values duplicated) per value in the array.

In [48]:
from pyspark.sql.functions import split, explode
df.withColumn("splitted", split(col("Description"), " "))\
.withColumn("exploded", explode(col("splitted")))\
.select("Description", "InvoiceNo", "exploded").show(10)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
|WHITE HANGING HEA...|   536365|   HEART|
|WHITE HANGING HEA...|   536365| T-LIGHT|
|WHITE HANGING HEA...|   536365|  HOLDER|
| WHITE METAL LANTERN|   536365|   WHITE|
| WHITE METAL LANTERN|   536365|   METAL|
| WHITE METAL LANTERN|   536365| LANTERN|
|CREAM CUPID HEART...|   536365|   CREAM|
|CREAM CUPID HEART...|   536365|   CUPID|
+--------------------+---------+--------+
only showing top 10 rows



**4) Maps**
Maps are created by using the map function and key-value pairs of columns. You then can select
them just like you might select from an array

In [49]:
from pyspark.sql.functions import create_map
df.select(create_map(col("InvoiceNo"), col("Description"))\
          .alias("Complex_map")).show(2)

+--------------------+
|         Complex_map|
+--------------------+
|{536365 -> WHITE ...|
|{536365 -> WHITE ...|
+--------------------+
only showing top 2 rows



In [52]:
df.select(create_map(col("InvoiceNo"), col("Description"))\
          .alias("Complex_map"))\
            .selectExpr("Complex_map['536365']").show(2)

+--------------------+
| Complex_map[536365]|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
+--------------------+
only showing top 2 rows



In [55]:
# You can also explode map types, which will turn them into columns:
df.select(create_map(col("InvoiceNo"), col("Description"))\
          .alias("Complex_map"))\
.selectExpr("explode(Complex_map)").show(2)

+------+--------------------+
|   key|               value|
+------+--------------------+
|536365|WHITE HANGING HEA...|
|536365| WHITE METAL LANTERN|
+------+--------------------+
only showing top 2 rows

